In [22]:
import sys
import quandl
import pandas
import numpy as np
import tensorflow as tf

quandl.ApiConfig.api_key = 'RUQDgag1PxTWxCVRgiBB'

# stock ndexes
item_list = ["YAHOO/INDEX_GSPC","YAHOO/INDEX_DJI","YAHOO/INDEX_IXIC","YAHOO/INDEX_NYA","YAHOO/INDEX_XAX","YAHOO/INDEX_RUT","YAHOO/INDEX_VIX","YAHOO/INDEX_GDAXI","YAHOO/INDEX_FCHI","YAHOO/INDEX_STOXX50E","YAHOO/INDEX_BFX","YAHOO/INDEX_N225","YAHOO/INDEX_HSI","YAHOO/INDEX_STI","YAHOO/INDEX_AXJO","YAHOO/INDEX_AORD","YAHOO/INDEX_BSESN","YAHOO/INDEX_JKSE","YAHOO/INDEX_NZ50","YAHOO/INDEX_KS11","YAHOO/INDEX_TWII","YAHOO/INDEX_GSPTSE","YAHOO/INDEX_BVSP","YAHOO/INDEX_MXX","YAHOO/INDEX_MERV","YAHOO/INDEX_TA100"]
   
# function
def get_formatted_data(start_date, end_date, estimate_day):
    # get stock data
    for i in range(len(item_list)):
        data = quandl.get(item_list[i], start_date=start_date, end_date=end_date, transform="diff")
        data = data.loc[:,"Adjusted Close":"Adjusted Close"]
        data.columns = [item_list[i]]
        if i == 0:
            stock_df = data
        else:
            stock_df = pandas.concat([stock_df, data], axis=1)

    # get gold data and shift
    gold_df = quandl.get("WGC/GOLD_DAILY_KRW", start_date = start_date, end_date = end_date, transform="diff")
    shifted_gold_df = gold_df.shift(periods = -1 * estimate_day)

    # merge
    merged_df =  pandas.concat([stock_df, shifted_gold_df], axis=1)
    merged_df = merged_df.dropna(axis=1, how = "all")
    merged_df = merged_df.dropna(axis=0, how = "any")
    
    # change data into tensorflow sytle
    num_of_sample = len(merged_df)
    num_of_stocks = len(merged_df.columns) - 1
    merged_np = merged_df.as_matrix()
    stock_np = merged_np[:,:num_of_stocks]
    gold_np = merged_np[:,num_of_stocks:]

    return stock_np, gold_np

# parameter
estimate_day = 1

# get train data
train_start_date = "2014-01-01"
train_end_date = "2016-12-31"
stock_train, gold_train = get_formatted_data(train_start_date, train_end_date, estimate_day)
if len(stock_train) == 0:
    print("No Validation Data")
    exit()
else:
    print("Trainning Data: ", len(stock_train))
    
# get valdaition data
valid_start_date = "2016-01-01"
valid_end_date = "2016-02-20"
stock_valid, gold_valid = get_formatted_data(valid_start_date, valid_end_date, estimate_day)
if len(stock_valid) == 0:
    print("No Validation Data")
else:
    print("Validation Data: ", len(stock_valid))

('Trainning Data: ', 508)
('Validation Data: ', 20)


In [31]:
# train and validation
w = tf.Variable(tf.random_uniform([len(stock_train[0]), 1], -5000.0, 5000.0, dtype=tf.float64))
b = tf.Variable(tf.zeros([1,1], dtype=tf.float64))
y =  tf.add(tf.matmul(stock_train, w), b)

cost_function = tf.reduce_mean(tf.abs(y - gold_train))
optimizer = tf.train.GradientDescentOptimizer(10)
train = optimizer.minimize(cost_function)
diff = tf.subtract(tf.add(tf.matmul(stock_valid, w),b),gold_valid)
diff_mean = tf.abs(tf.reduce_mean(diff))

In [32]:
# tesnorflow
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(len(stock_train)):
        sess.run(train)
        if step % 10 == 0 or step == len(stock_valid) - 1:
            print("Step: %3d" % (step), "Reduce Mean: ", sess.run(diff_mean))
    print(sess.run(diff))

('Step:   0', 'Reduce Mean: ', 266345.72264010319)
('Step:  10', 'Reduce Mean: ', 140009.13761295966)
('Step:  19', 'Reduce Mean: ', 270751.28296506591)
('Step:  20', 'Reduce Mean: ', 95589.138976837698)
('Step:  30', 'Reduce Mean: ', 64324.719987164252)
('Step:  40', 'Reduce Mean: ', 41644.040408177396)
('Step:  50', 'Reduce Mean: ', 17516.788219660801)
('Step:  60', 'Reduce Mean: ', 1328.777990197204)
('Step:  70', 'Reduce Mean: ', 12646.018702928162)
('Step:  80', 'Reduce Mean: ', 25577.860285993851)
('Step:  90', 'Reduce Mean: ', 36869.590495600831)
('Step: 100', 'Reduce Mean: ', 42677.15803027209)
('Step: 110', 'Reduce Mean: ', 47562.93565128334)
('Step: 120', 'Reduce Mean: ', 47048.417062476183)
('Step: 130', 'Reduce Mean: ', 54558.984898344614)
('Step: 140', 'Reduce Mean: ', 57563.720270906299)
('Step: 150', 'Reduce Mean: ', 61252.053503867428)
('Step: 160', 'Reduce Mean: ', 63075.057062099593)
('Step: 170', 'Reduce Mean: ', 66297.038903765293)
('Step: 180', 'Reduce Mean: ', 679